In [25]:
import jax 
import jax.numpy as jnp
import numpy as np 
import pandas as pd
from cluster import BasicClusteringRecommender, KMeansRecommender

In [26]:
train_df = pd.read_csv('../preprocessed_dataset/train_dataset.csv')
test_df = pd.read_csv('../preprocessed_dataset/test_dataset.csv')

In [27]:
train_df.head()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
0,0,1179720,815,10,115603,5,186,17212
1,1,1037453,620,10,65510,5,138,9767
2,2,103387,11415,10,74460,5,2402,11085
3,3,895146,4255,10,7294,5,910,1125
4,4,305030,14171,10,128621,5,2988,19123


In [28]:
train_df.describe()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
count,40000.00000,4.000000e+04,40000.000000,40000.0,40000.000000,40000.000000,40000.000000,40000.000000
mean,19999.50000,6.629283e+05,11816.062275,10.0,67869.195175,4.548675,2499.701775,10121.717800
std,11547.14972,3.501890e+05,6833.546813,0.0,39498.025905,0.695265,1442.664511,5859.528774
min,0.00000,0.000000e+00,0.000000,10.0,2.000000,3.000000,0.000000,0.000000
25%,9999.75000,3.773340e+05,5806.000000,10.0,33787.250000,4.000000,1247.000000,5084.750000
50%,19999.50000,7.340415e+05,11853.000000,10.0,67459.500000,5.000000,2501.000000,10064.500000
75%,29999.25000,9.181072e+05,17743.000000,10.0,101899.000000,5.000000,3748.000000,15193.000000
max,39999.00000,1.315057e+06,23680.000000,10.0,136729.000000,5.000000,4999.000000,20308.000000


In [29]:
test_df.describe()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
count,6551.000000,6.551000e+03,6551.000000,6551.0,6551.000000,6551.000000,6551.000000,6551.000000
mean,5003.284689,6.727516e+05,11839.409098,10.0,68064.745535,4.565715,2504.632117,10150.780034
std,2887.366461,3.497481e+05,6833.804936,0.0,39461.786362,0.685672,1442.709450,5853.983402
min,0.000000,3.300000e+01,17.000000,10.0,69.000000,3.000000,1.000000,9.000000
25%,2498.000000,3.898040e+05,5896.000000,10.0,34241.000000,4.000000,1265.000000,5143.000000
50%,5012.000000,7.354160e+05,11909.000000,10.0,67818.000000,5.000000,2514.000000,10118.000000
75%,7490.500000,9.287125e+05,17723.500000,10.0,102331.000000,5.000000,3743.500000,15254.500000
max,9999.000000,1.315058e+06,23680.000000,10.0,136719.000000,5.000000,4999.000000,20307.000000


In [30]:
train_df = train_df[['rating', 'user_id_idx', 'item_id_idx']]
test_df = test_df[['rating', 'user_id_idx', 'item_id_idx']]

In [31]:
train_df.rename(columns={'user_id_idx': 'userID', 'item_id_idx': 'songID'}, inplace=True)
test_df.rename(columns={'user_id_idx': 'userID', 'item_id_idx': 'songID'}, inplace=True)

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score

def eval_model(model, test_df):
    predicted_rating = test_df.apply(lambda row: model.predict_score(row.userID, row.songID), axis=1)
    precision = precision_score(test_df.rating, predicted_rating, average='macro')
    recall = recall_score(test_df.rating, predicted_rating, average='macro')
    f1 = f1_score(test_df.rating, predicted_rating, average='macro')

    # print(predicted_rating.describe())

    return precision, recall, f1

**Evaluate Basic Clustering Recommender**

In [33]:
recommender = BasicClusteringRecommender(n_neighbor=200)

In [34]:
recommender.fit(train_df)

In [35]:
eval_model(recommender, test_df.iloc[:100])
# May take ~2 hours to evaluate all the test cases

(0.3511011011011011, 0.34150326797385616, 0.19122234826999432)

**Evaluate K-Means Recommdender with Minibatch**

In [36]:
recommender = KMeansRecommender(n_neighbor=200, use_minibatch=True)

In [37]:
recommender.fit(train_df)

In [38]:
eval_model(recommender, test_df.iloc[:100])

(0.35964912280701755, 0.33660130718954245, 0.18452380952380953)

**Evaluate K-Means Recommender without Minibatch**

In [39]:
recommender = KMeansRecommender(n_neighbor=200, use_minibatch=False)

In [40]:
recommender.fit(train_df)

In [41]:
eval_model(recommender, test_df.iloc[:100])

(0.3189102564102564, 0.4011437908496733, 0.17153931339977854)